## MQTT Receiver

In [ ]:
"""
!pip install redis
!pip install paho-mqtt
"""

In [ ]:
import uuid
import redis
import paho.mqtt.client as mqtt
import json
from datetime import datetime

In [ ]:
sensor = input("Insert current sensor name:")
clean = input("Do you want to clean timeseries before (y/n):")


### Initialize Redis Client

In [ ]:
redis_host = 'redis-19881.c268.eu-west-1-2.ec2.cloud.redislabs.com'
redis_port = 19881
redis_password = 'Z3uevMG7GPbr5g66RbiGFWrQhsgKLdfO'

redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password)

try:
    is_connected = redis_client.ping()
    print('Redis client is connected ', is_connected)
except Exception as ex:
    print(ex)

def push_data(listening_to: str, probability: int, tmp: int, sensor: str):

    listening = 0
    if listening_to == "speech":
        listening = -1
    elif listening_to == "music":
        listening = 1
    redis_client.ts().add(f'listening_to_' + sensor, int(tmp), listening)
    redis_client.ts().add(f'probability_' + sensor, int(tmp), probability)

def clean_timeseries(timeseries_key: str):
    try:
        redis_client.delete(timeseries_key)
    except:
        print("Timeseries does not exist")

def init_timeseries(timeseries_key: str):
    try:
        redis_client.ts().create(timeseries_key)
    except:
        print("Timeseries already exists")

if clean == "y":
    clean_timeseries("listening_to_" + sensor)
    clean_timeseries("probability_" + sensor)
    
init_timeseries("listening_to_" + sensor)
init_timeseries("probability_" + sensor)

mac_address = hex(uuid.getnode())

Redis client is connected  True


### Initialize MQTT Client

In [ ]:
TOPIC = 's290453/sensor_data/+'
BROKER = 'hermesdmpmqttbroker.e3ebhhb8bkhhd6fh.westeurope.azurecontainer.io'
PORT = 1883
KEEP_ALIVE = 60


def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe(TOPIC)


def on_message(client, userdata, msg):
    print(msg.topic + " " + str(msg.payload))
    sensor = msg.topic.split('/')[2]
    print(sensor)
    try:
        data = json.loads(msg.payload)
        datetime_format = "%Y-%m-%d %H:%M:%S"
        timestamp = datetime.strptime(data["timestamp"], datetime_format)
        ts = timestamp.timestamp()
        listening_to = data["listening_to"]
        probability = data["probability"]
        push_data(listening_to=listening_to, probability=probability, tmp=ts, sensor=sensor)
    except Exception as ex:
        print(ex)


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.connect(BROKER, PORT, KEEP_ALIVE)

client.loop_forever()

s290453/sensor_data/s5 b'{"listening_to": "music", "probability": 0.8918551355156158, "timestamp": "2023-07-10 17:59:19"}'
s5
s290453/sensor_data/s5 b'{"listening_to": "music", "probability": 0.7074672294764186, "timestamp": "2023-07-10 17:59:22"}'
s5
s290453/sensor_data/s5 b'{"listening_to": "music", "probability": 0.9519653328680736, "timestamp": "2023-07-10 17:59:27"}'
s5
s290453/sensor_data/s5 b'{"listening_to": "music", "probability": 0.839490819960144, "timestamp": "2023-07-10 17:59:30"}'
s5
s290453/sensor_data/s5 b'{"listening_to": "music", "probability": 0.7728336718653303, "timestamp": "2023-07-10 17:59:36"}'
s5
s290453/sensor_data/s5 b'{"listening_to": "music", "probability": 0.9232647362794664, "timestamp": "2023-07-10 17:59:39"}'
s5
s290453/sensor_data/s5 b'{"listening_to": "music", "probability": 0.8128843758275404, "timestamp": "2023-07-10 17:59:44"}'
s5
s290453/sensor_data/s5 b'{"listening_to": "music", "probability": 0.7923690632909599, "timestamp": "2023-07-10 17:59:49

KernelInterrupted: Execution interrupted by the Jupyter kernel.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1e190a0d-e0c4-4d69-ad9e-7a1f03e6c992' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>